Create Failover Cluster
==================================

Description
-----------

Notebook to walk through creating a failover cluster on on-premises SQL Server Database.

Note: before running the steps below, make sure you have the following:

- A resource group with a domain controller.
- One or more domain joined VMs in Azure running SQL Server 2016 (or later) in the _same_ availability set or _different_ availability zones that have been registered with the SQL IaaS Agent extension
- Two freely available (not used) IP addresses respectively for the internal load balancer and availability group listener in the same subnet as the availability group. If you use an existing load balancer, you only need the IP for the availability group listener.

Steps of this procedure include:

1. Connect to Azure subscription
2. Provision resource group for SQL VM migration
3. Create a storage account
4. Define cluster metadata
5. Add VMs to the cluster
6. Validate cluster

In [ ]:
#Set Parameters
$ClusterName = ""                        # The name of the SQL VM Cluster.
$SqlVm1Name = ""                         # The name of the first SQL VM.
$SqlVm2Name = ""                         # The name of the second SQL VM.
$Location = ""                           # Specify the valid Location such as 'West US 2','EASTUS' etc...
$PublisherName = ""                      # Name of Publisher, Default would be 'MicrosoftSQLServer'
$ResourceGroupName = ""                      # Resource group name(It should be of alphabets case insensitive)
$SkuName = ""                            # Specify the valid SKU...Such as 'Standard_LRS'
$StorageAccountName = ""                 # Name of Storage Account used for cloud witness.

#Add VMs to the cluster
$OperatorAccountPassword = ""            # Password for Operator Account
$ServiceAccountPassword = ""             # Password for Service Account
$BootstrapAccountPassword = ""           # Password for Bootstrap Account

#Define Cluster Media
$FQDN = "www.microsoft.com"              # Domain name, such as www.microsoft.com.
$OperatorAccount = ""                    # Domain account for Operator
$BootstrapAccount = ""                   # Domain account for Bootstrap
$ServiceAccount = ""                     # Service account

$SqlPath = "sqlserver:\sql\$($env:COMPUTERNAME)"      #This script will generate Sql Path

### <b>Sign in to Azure </b>
Sign in to your Azure Subscription with the _Connect-AzAccount_ command and follow the on-screen directions.

In [ ]:
Connect-AzAccount

### <b>Get Subscription</b>
Below command will open a _**Dialouge Box**_ with list of subscriptions. Selecting one of those will set that Subscription for rest of the commands.

In [ ]:
$Subscription = Get-AzSubscription | Out-GridView -PassThru
Set-AzContext -SubscriptionName $Subscription

_If you don't know which Location you want to use, you can list the available Locations. Display the list of Locations by using the following code example and find the one you want to use. This example uses eastus. Store the Location in a variable and use the variable so you can change it in one place._

In [ ]:
Get-AzLocation | select Location
$Location


### <b>Create a resource group</b>
Create an Azure resource group with _New-AzResourceGroup_. A resource group is a logical container into which Azure resources are deployed and managed.

In [ ]:
# Create Azure resource group, if necessary
$ResourceGroup = Get-AzResourceGroup -Name $ResourceGroupName

if (!$ResourceGroup)
{
    # Need to create a new resource group
    Write-Output "Resource Group $ResourceGroupName does not exist. Creating..."
    $ResourceGroup = New-AzResourceGroup -Name $ResourceGroupName -Location $Location
}

### <b>Create a storage account</b>
Create a standard, general-purpose storage account with LRS replication by using _New-AzStorageAccount_. Next, get the storage account context that defines the storage account you want to use. When acting on a storage account, reference the context instead of repeatedly passing in the credentials. Use the following example to create a storage account called storageaccountazure with locally redundant storage (LRS) and blob encryption (enabled by default).

In [ ]:
$StorageAccount = New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name $StorageAccountName `
    -SkuName $SkuName -Location $Location -Kind StorageV2 `
    -AccessTier Hot -EnableHttpsTrafficOnly $true

$Ctx = $StorageAccount.Context

### <b>Create a container</b>
Blobs are always uploaded into a container. You can organize groups of blobs like the way you organize your files on your computer in folders.
Set the container name, then create the container by using _New-AzStorageContainer_. Set the permissions to blob to allow public access of the files. The container name in this example is quickstartblobs.

In [ ]:
$ContainerName = "quickstartblobs"
New-AzStorageContainer -Name $ContainerName -Context $Ctx -Permission blob

### <b>Get Azure Storage Acount Key</b>
This script will get the key for Storage Account which is been created.

In [ ]:
$StorageAccountKey = `
    (Get-AzStorageAccountKey `
    -ResourceGroupName $ResourceGroupName `
    -Name $StorageAccountName).Value[0]

### <b>Get Azure Storage Container Uri</b>
The following script can be used to get the Uri of Storage container.

In [ ]:
    $StorageUri = (Get-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name $StorageAccountName | Get-AzStorageContainer | Where-Object { $_.Name -eq $AzureContainerName }).CloudBlobContainer.Uri.AbsoluteUri 

### **Azure SQL VM Offers**

Run the following command to get updated list of offers for Microsoft SQL Server in your location. These settings can be used for the OfferName associated with this migration. Configure it for your purposes.

Note that the SQL Version is first then appended with an operating system version. E.g.: "WS2019" means Windows Server 2019. Along with various versions of Windows Servers, there are also enterprise Linux versions such as RedHat Enterprise, Suse Enterprise, and Ubuntu. Some versions are BYOL (Bring Your Own License) aka [Hybrid Benefit](https://azure.microsoft.com/en-us/pricing/hybrid-benefit/).

In [ ]:
$Offer = Get-AzVMImageOffer -Location $Location -Publisher $PublisherName | Select Offer

### **Define cluster metadata**

The following script is used to define the metadata for Windows Server Failover Cluster so that when the first SQL Server VM is added, the cluster is created as defined.

In [ ]:
$StorageAccountUrl = "https://$StorageAccountName.blob.core.windows.net/"

$group = New-AzSqlVMGroup -Name $ClusterName -Location $Location `
  -ResourceGroupName $ResourceGroupName -Offer "SQL2017-WS2016" `
  -Sku Enterprise -DomainFqdn $FQDN -ClusterOperatorAccount $OperatorAccount `
  -ClusterBootstrapAccount $BootstrapAccount  -SqlServiceAccount $ServiceAccount `
  -StorageAccountUrl $StorageAccountUrl `
  -StorageAccountPrimaryKey $StorageAccountKey

### **Add VMs to the cluster**

Adding the first SQL Server VM to the cluster creates the cluster. The <mark>Update-AzSqlVM</mark> command creates the cluster with the name previously given, installs the cluster role on the SQL Server VMs, and adds them to the cluster. Subsequent uses of the command adds more SQL Server VMs to the newly created cluster.

In [ ]:
$sqlvm1 = Get-AzSqlVM -Name $SqlVm1Name -ResourceGroupName $ResourceGroup
$sqlvm2 = Get-AzSqlVM -Name $SqlVm2Name -ResourceGroupName $ResourceGroup
$SecureOAPassword = ConvertTo-SecureString $OperatorAccountPassword -AsPlainText -Force
$SecureSAPassword = ConvertTo-SecureString $ServiceAccountPassword -AsPlainText -Force
$SecureBAPassword = ConvertTo-SecureString $BootstrapAccountPassword -AsPlainText -Force

$sqlvmconfig1 = Set-AzSqlVMConfigGroup -SqlVM $sqlvm1 `
   -SqlVMGroup $group -ClusterOperatorAccountPassword $SecureOAPassword `
   -SqlServiceAccountPassword $SecureSAPassword `
   -ClusterBootstrapAccountPassword $SecureBAPassword

Update-AzSqlVM -ResourceId $sqlvm1.ResourceId -SqlVM $sqlvmconfig1

$sqlvmconfig2 = Set-AzSqlVMConfigGroup -SqlVM $sqlvm2 `
   -SqlVMGroup $group -ClusterOperatorAccountPassword $SecureOAPassword `
   -SqlServiceAccountPassword $SecureSAPassword `
   -ClusterBootstrapAccountPassword $SecureBAPassword

Update-AzSqlVM -ResourceId $sqlvm2.ResourceId -SqlVM $sqlvmconfig2

### **Validate cluster**

For a failover cluster to be supported by Microsoft, it must pass cluster validation. Failure to do so leaves your cluster in an unsupported state.

In [ ]:
Test-Cluster –Node ($SqlVm1Name,$SqlVm2Name) –Include "Inventory", "Network", "System Configuration"